# Section 4 — Filtering, Sorting, and Conditional Selection

### Goal
Efficiently extract relevant rows and columns from large datasets using Pandas' expressive filtering and sorting capabilities.
By the end of this section, you will:
- Use boolean masks and conditional filters.
- Apply multi-condition filtering with `&` and `|`.
- Sort values and indexes.
- Use `query()` and `isin()` for cleaner syntax.
- Combine filters for real-world decision logic.

## Subtopics
1. Boolean masking and logical operators.
2. Filtering with `query()` and `isin()`.
3. Sorting rows and indexes.
4. Chained filtering and conditional updates.
5. Real-world filtering tasks and performance notes.

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(10)

# Simulate a retail transactions dataset
data = {
    'CustomerID': range(1, 11),
    'Gender': np.random.choice(['Male', 'Female'], 10),
    'Age': np.random.randint(18, 65, 10),
    'Region': np.random.choice(['North', 'South', 'East', 'West'], 10),
    'PurchaseAmount': np.random.randint(50, 1000, 10)
}
df = pd.DataFrame(data)
df

## 1️⃣ Boolean Masking Basics

Boolean masks are arrays of True/False values used to filter rows. You can directly pass comparisons to the DataFrame:

In [ ]:
# Customers older than 40
mask = df['Age'] > 40
df[mask]

### Combining Conditions
- Use `&` for **AND**, `|` for **OR**, and `~` for **NOT**.
- Always enclose each condition in parentheses!

In [ ]:
# Female customers in the South region older than 30
filtered = df[(df['Gender'] == 'Female') & (df['Region'] == 'South') & (df['Age'] > 30)]
filtered

## 2️⃣ Filtering with `isin()` and `query()`

`isin()` is ideal for membership filtering (e.g., multiple regions). `query()` offers a readable, SQL-like syntax for complex filters.

In [ ]:
# Using isin()
target_regions = ['North', 'West']
df[df['Region'].isin(target_regions)]

In [ ]:
# Using query()
df.query('PurchaseAmount > 500 and Gender == "Male"')

## 3️⃣ Sorting Data

Use `sort_values()` for sorting rows and `sort_index()` for index ordering.

In [ ]:
# Sort by PurchaseAmount descending
df_sorted = df.sort_values(by='PurchaseAmount', ascending=False)
df_sorted

In [ ]:
# Multi-column sort: Region (A–Z) then Age (descending)
df_sorted_multi = df.sort_values(by=['Region', 'Age'], ascending=[True, False])
df_sorted_multi

## 4️⃣ Conditional Updates and Derived Columns
Apply transformations selectively using boolean masks or `np.where()`.

In [ ]:
# Tag high-value customers (>=700)
df['HighValue'] = np.where(df['PurchaseAmount'] >= 700, 'Yes', 'No')
df

In [ ]:
# Give 5% discount to customers from East region
df.loc[df['Region'] == 'East', 'PurchaseAmount'] *= 0.95
df

## ✅ Real-World Problem 1 — Customer Segmentation

**Scenario:** You manage a retail dataset and want to find premium customers.
- Filter those with purchases above the 75th percentile.
- Restrict to regions 'North' and 'West'.
- Sort by PurchaseAmount descending.

In [ ]:
threshold = df['PurchaseAmount'].quantile(0.75)
premium = df[(df['PurchaseAmount'] >= threshold) & (df['Region'].isin(['North', 'West']))]
premium.sort_values('PurchaseAmount', ascending=False)

## ✅ Real-World Problem 2 — Detecting At-Risk Customers

**Goal:** Identify customers with low spending (<200) or from underperforming regions.

1. Define low-spend threshold.
2. Mark these customers.
3. Output a summary count by region.

In [ ]:
low_threshold = 200
df['AtRisk'] = np.where((df['PurchaseAmount'] < low_threshold) | (df['Region'].isin(['East'])), 'Yes', 'No')
df.groupby('Region')['AtRisk'].value_counts()

## ⚙️ Under the Hood: How Pandas Filtering Works
- Boolean masks are converted to NumPy boolean arrays and used for fast indexing.
- `query()` leverages `numexpr` (if available) to compile expressions to bytecode for vectorized evaluation.
- `isin()` uses hash-based membership lookups for O(n) efficiency.
- Sorting uses Timsort (same as Python) — stable and adaptive for real-world data distributions.

## 💡 Best Practices / Common Pitfalls
- Always use parentheses when combining multiple conditions.
- Avoid chained indexing like `df[df['A']>0]['B']=...` — use `.loc[]` instead.
- For large data, prefer `query()` or `eval()` to reduce memory overhead.
- Avoid sorting in loops — sort once, reuse order.
- Don’t compare strings directly without trimming/lowercasing first.

## 🧩 Challenge Exercise

**Dataset:** Suppose you have a DataFrame `sales` with columns `['Region', 'SalesRep', 'MonthlySales', 'Target', 'Month']`.

**Tasks:**
1. Filter reps who achieved more than 90% of their target.
2. Exclude regions `'East'` and `'South'`.
3. Add a new column `Performance`:
   - `'Excellent'` if `MonthlySales >= 1.2 * Target`
   - `'Good'` if between 0.9–1.2
   - `'Needs Improvement'` otherwise.
4. Sort by performance category and descending MonthlySales.

_Hint:_ Use `np.select()` for multi-condition assignment and chained sorting with custom order.

# --- End of Section 4 — Continue to Section 5 ---